<a href="https://colab.research.google.com/github/pratikbhongade/AchPub/blob/main/ACHProjectKey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Open the ACH file and read the lines
with open('testnacha.ach', 'r') as f:
    lines = f.readlines()

# Create a DataFrame for each record type
header_df = pd.DataFrame([{
    'Record Type': lines[0][0:1],
    'Priority Code': lines[0][1:3],
    'Immediate Dest': lines[0][3:13],
    'Immediate Origin': lines[0][13:23],
    'Creation Date': lines[0][23:29],
    'Creation Time': lines[0][29:33],
    'File ID Modifier': lines[0][33:34],
    'Record Size': lines[0][34:37],
    'Blocking Factor': lines[0][37:39],
    'Format Code': lines[0][39:40],
    'Immediate Dest Name': lines[0][40:63],
    'Immediate Origin Name': lines[0][63:86],
    'Reference Code': lines[0][86:94]
}], index=[0])

# Create a DataFrame to store the batch header record entries
batch_header_df = pd.DataFrame(columns=['Record Type', 'Service Class Code', 'Company Name',
                                        'Company Identification', 'Standard Entry Class Code',
                                        'Company Entry Description', 'Company Descriptive Date',
                                        'Effective Entry Date', 'Settlement Date', 'Originator Status Code',
                                        'Originating DFI Identification', 'Batch Number'])

# Create a DataFrame to store the entries
entries_df = pd.DataFrame(columns=['Record Type', 'Transaction Code', 'Receiving DFI Identification',
                                   'Check Digit', 'DFI Account Number', 'Amount', 'Individual ID Number',
                                   'Individual Name', 'Discretionary Data', 'Addenda Indicator', 'Trace Number'])

# Iterate over the lines in the file and populate the DataFrames
for i in range(0, len(lines), 1):
    if lines[i][0:1] == '5':
        batch_header_df = batch_header_df.append({
            'Record Type': lines[i][0:1],
            'Service Class Code': lines[i][1:4],
            'Company Name': lines[i][4:20],
            'Company Discretionary Data': lines[i][20:40],
            'Company Identification': lines[i][40:50],
            'Standard Entry Class Code': lines[i][50:53],
            'Company Entry Description': lines[i][53:63],
            'Company Descriptive Date': lines[i][63:69],
            'Effective Entry Date': lines[i][69:75],
            'Settlement Date': lines[i][75:81],
            'Originator Status Code': lines[i][81:82],
            'Originating DFI Identification': lines[i][82:92],
            'Batch Number': lines[i][94:100]
        }, ignore_index=True)

entries_df = pd.DataFrame(columns=['Record Type', 'Transaction Code', 'Receiving DFI Identification', 'Check Digit', 'DFI Account Number', 'Amount', 'Individual ID Number', 'Individual Name', 'Discretionary Data', 'Addenda Indicator', 'Trace Number'])

for i in range(1, len(lines) - 1, 1):
    if lines[i][0:1] == '6':
        entries_df = entries_df.append({
            'Record Type': lines[i][0:1],
            'Transaction Code': lines[i][1:3],
            'Receiving DFI Identification': lines[i][3:13],
            'Check Digit': lines[i][13:14],
            'DFI Account Number': lines[i][14:29],
            'Amount': lines[i][29:39],
            'Individual ID Number': lines[i][39:54],
            'Individual Name': lines[i][54:76],
            'Discretionary Data': lines[i][76:78],
            'Addenda Indicator': lines[i][78:79],
            'Trace Number': lines[i][79:94]
        }, ignore_index=True)


# Initialize Batch Control Record table
# Batch Control Record
# Batch Control Record
# Create empty dataframes for batch header, entries, and control records
# Batch Control Record
batch_control_df = pd.DataFrame(columns=['record_type', 'service_class_code', 'entry_count', 'entry_hash',
                                          'total_debit_amount', 'total_credit_amount', 'company_id',
                                          'message_authentication_code', 'reserved'])
batch_count = 0
for line in lines:
    record_type = line[0]
    if record_type == '5':
        batch_count += 1
    elif record_type == '8':
        batch_control_df = batch_control_df.append({
            'record_type': '8',
            'service_class_code': line[1:4],
            'entry_count': line[4:10],
            'entry_hash': line[10:20],
            'total_debit_amount': line[20:31],
            'total_credit_amount': line[31:44],
            'company_id': line[44:53],
            'message_authentication_code': line[53:69],
            'reserved': line[69:94]
        }, ignore_index=True)
        if batch_count == 1:
            batch_control_df.iloc[-1, 0] = '8'
            batch_count = 0

# Remove the last row if it is empty
if batch_control_df.iloc[-1].isnull().all():
    batch_control_df = batch_control_df.iloc[:-1]

file_control_df = pd.DataFrame([{
    'record_type': lines[-1][0:1],
    'batch_count': lines[-1][1:7],
    'block_count': lines[-1][7:13],
    'entry_count': lines[-1][13:21],
    'entry_hash': lines[-1][21:32],
    'total_debit_amount': lines[-1][32:43],
    'total_credit_amount': lines[-1][43:55],
    'reserved': lines[-1][56:94]
}])

from tabulate import tabulate 

total_credit_amount = 0
for index, row in batch_control_df.iterrows():
    credit_amount = float(row['total_credit_amount']) / 100
    total_credit_amount += credit_amount

batch1_credit_amount = float(batch_control_df.loc[0, 'total_credit_amount']) / 100
batch2_credit_amount = float(batch_control_df.loc[1, 'total_credit_amount']) / 100
sum_credit_amount = batch1_credit_amount + batch2_credit_amount

file_control_credit_amount = float(file_control_df.iloc[0]['total_credit_amount']) / 100

if total_credit_amount == file_control_credit_amount:
    ach_validity = 'Valid'
else:
    ach_validity = 'Invalid'

# Print results in tabular format
table_headers = ['Batch Details', 'Credit Amount']
table_data = [['Batch 1', batch1_credit_amount],
              ['Batch 2', batch2_credit_amount],
              ['Sum of Batch 1 and Batch 2', sum_credit_amount],
              ['Total Credit Ammount', file_control_credit_amount],
              ['ACH Validity', ach_validity]]

print(tabulate(table_data, headers=table_headers, tablefmt='fancy_grid'))


ach_table_headers = ['ACH Validity','Result']
ach_table_data = [[f.name ,ach_validity]]
print(tabulate(ach_table_data, headers=ach_table_headers, tablefmt='fancy_grid'))




print("Header Record\n")
print(tabulate(header_df, headers='keys', tablefmt='fancy_grid'))

# Print the batch header DataFrame
print("\nBatch Header Record\n")
print(tabulate(batch_header_df, headers='keys', tablefmt='fancy_grid'))

# Print the entries DataFrame
print("\nEntries\n")
print(tabulate(entries_df, headers='keys', tablefmt='fancy_grid'))

print("\nBatch Control Record\n")
print(tabulate(batch_control_df, headers='keys', tablefmt='fancy_grid'))

print("\nFile Control Record\n")
print(tabulate(file_control_df, headers='keys', tablefmt='fancy_grid'))


╒════════════════════════════╤═════════════════╕
│ Batch Details              │ Credit Amount   │
╞════════════════════════════╪═════════════════╡
│ Batch 1                    │ 21750.75        │
├────────────────────────────┼─────────────────┤
│ Batch 2                    │ 22200.25        │
├────────────────────────────┼─────────────────┤
│ Sum of Batch 1 and Batch 2 │ 43951.0         │
├────────────────────────────┼─────────────────┤
│ Total Credit Ammount       │ 43952.0         │
├────────────────────────────┼─────────────────┤
│ ACH Validity               │ Invalid         │
╘════════════════════════════╧═════════════════╛
╒════════════════╤══════════╕
│ ACH Validity   │ Result   │
╞════════════════╪══════════╡
│ testnacha.ach  │ Invalid  │
╘════════════════╧══════════╛
Header Record

╒════╤═══════════════╤═════════════════╤══════════════════╤════════════════════╤═════════════════╤═════════════════╤════════════════════╤═══════════════╤═══════════════════╤═══════════════╤═════════

<ipython-input-1-9ff9bc51ae54>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch_header_df = batch_header_df.append({
<ipython-input-1-9ff9bc51ae54>:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch_header_df = batch_header_df.append({
<ipython-input-1-9ff9bc51ae54>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  entries_df = entries_df.append({
<ipython-input-1-9ff9bc51ae54>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  entries_df = entries_df.append({
<ipython-input-1-9ff9bc51ae54>:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
